In [1]:
import pywren
import pickle 
import time
import boto3
import numpy as np

def latency(n):
    s3_client = boto3.client('s3')
    bucket_name = 'test-yawen'
    
    random_num = np.random.rand(1, n) #generate n floats (each becomes 20B after serializing)
    rand_serial = pickle.dumps(random_num)
    t0 = time.time()
    for i in range(100):
        s3_client.put_object(
            Bucket = bucket_name,
            Body = rand_serial,
            Key = 'tmp'+str(n)+'/'+str(i)
        )   
    t1 = time.time()
    for i in range(100):
        s3_client.get_object(Bucket=bucket_name, Key='tmp'+str(n)+'/'+str(i))['Body'].read()
    t2 = time.time()
    return (n, (t1-t0)*10, (t2-t1)*10) # time in ms 

In [10]:
wrenexec = pywren.default_executor()

In [11]:
future = wrenexec.call_async(latency, 1)
future.result()

(1, 20.595228672027588, 13.582830429077148)

In [12]:
future = wrenexec.call_async(latency, 10)
future.result()

(10, 28.280160427093506, 10.745761394500732)

In [13]:
future = wrenexec.call_async(latency, 100)
future.result()

(100, 22.29712963104248, 11.682398319244385)